In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from src.models import EndiannessModel
from src.dataset_loaders import MipsMipselDataset, random_train_test_split
from src.transforms import EndiannessCount

num_epochs = 2
learning_rate = 0.001
batch_size = 4
SEED = 42

# Create dataloaders
mips_dir = "../../dataset/ISAdetect/ISAdetect_full_dataset/mips"
mipsel_dir = "../../dataset/ISAdetect/ISAdetect_full_dataset/mipsel"

dataset = MipsMipselDataset(mips_dir, mipsel_dir, transform=EndiannessCount())
train_set, test_set = random_train_test_split(dataset, test_split=0.2, seed=SEED)

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=1)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=1)

In [2]:
model = EndiannessModel(with_sigmoid=False)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print("Training model...")
# Training loop
for epoch in range(num_epochs):
    print(f"before enumerate {epoch}")
    for i, (batch_x, batch_y) in enumerate(train_loader):
        if i == 0:
            print(f"starting_epoch {epoch}")
        optimizer.zero_grad()
        output = model(batch_x)
        targets = batch_y.unsqueeze(1)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()

        # if epoch == 0 and i < 10:
        #     print(f"Initial loss: {loss.item()}")

        if (i + 1) % (len(train_loader) // 10) == 0:
            print(f"Step {i+1}, Loss: {loss.item()}")
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

Training model...
before enumerate 0
starting_epoch 0
Step 146, Loss: 0.11894682049751282
Step 292, Loss: 0.000629525980912149
Step 438, Loss: 0.004226872231811285
Step 584, Loss: 0.007108584977686405
Step 730, Loss: 0.03126688301563263
Step 876, Loss: 0.0006168913678266108
Step 1022, Loss: 0.031408365815877914
Step 1168, Loss: 0.0
Step 1314, Loss: 0.00620176550000906
Step 1460, Loss: 0.0059243785217404366
Epoch 1/2, Loss: 0.0005811500595882535
before enumerate 1
starting_epoch 1
Step 146, Loss: 0.0011509942123666406
Step 292, Loss: 0.006798191461712122
Step 438, Loss: 0.00038724931073375046
Step 584, Loss: 2.0265497369109653e-06
Step 730, Loss: 0.0038630468770861626
Step 876, Loss: 0.005509813316166401
Step 1022, Loss: 5.92399992456194e-05
Step 1168, Loss: 0.007289537228643894
Step 1314, Loss: 0.001799143967218697
Step 1460, Loss: 0.0005013977643102407
Epoch 2/2, Loss: 0.0


In [3]:
# Test model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    class_1 = 0
    for batch_x, batch_y in test_loader:
        output = model(batch_x)
        for i in range(len(output)):
            if output[i] >= 0.5:
                pred = 1
            else:
                pred = 0
            if pred == batch_y[i]:
                correct += 1
            if batch_y[i] == 1:
                class_1 += 1
            total += 1
    print(f"Accuracy: {correct/total}")
    print(f"Class 1: {class_1}, Total: {total}, Percentage: {class_1/total}")

Accuracy: 0.9986376021798365
Class 1: 761, Total: 1468, Percentage: 0.5183923705722071
